<a href="https://colab.research.google.com/github/hndrr/colab/blob/main/kandinsky2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 引き当てたGPUを確認
!nvidia-smi

In [ ]:
!pip install 'git+https://github.com/ai-forever/Kandinsky-2.git'
!pip install git+https://github.com/openai/CLIP.git
!pip install gradio

In [ ]:
from kandinsky2 import get_kandinsky2
model = get_kandinsky2('cuda', task_type='text2img', cache_dir='/tmp/kandinsky2', model_version='2.1', use_flash_attention=False)

In [ ]:
import gradio as gr

def infer(prompt):
    images = model.generate_text2img(prompt, num_steps=100,
                          batch_size=1, guidance_scale=4,
                           h=768, w=768,
                           sampler='p_sampler', prior_cf_scale=4,
                           prior_steps="5",)
    return images

with gr.Blocks() as demo:
    with gr.Group():
        with gr.Box():
            with gr.Row().style(equal_height=True):

                text = gr.Textbox(
                    label="Enter your prompt", show_label=False, max_lines=1, placeholder="プロンプトを入力"
                )
                btn = gr.Button("Generate")

        gallery = gr.Gallery(label="Generated images", show_label=False, elem_id="generated_id")

        # text.submit(infer, inputs=[text], outputs=gallery)
        btn.click(infer, inputs=[text], outputs=gallery)

demo.queue(max_size=15).launch()

In [ ]:
# 別のUI
# 参考
# https://gradio.app/quickstart/
# https://bounote.jp/category/programming/gradio/

import gradio as gr

def infer(prompt):
    images = model.generate_text2img(prompt, num_steps=100,
                          batch_size=1, guidance_scale=4,
                           h=768, w=768,
                           sampler='p_sampler', prior_cf_scale=4,
                           prior_steps="5",)
    return images[0]

iface = gr.Interface(
  fn=infer, 
  inputs='text',
  outputs='image',
  examples=[["赤い猫"]],
  allow_flagging='never'
)

iface.launch()